# Hierarchical optimization with relative data

In this notebook we illustrate how to do hierarchical optimization in pyPESTO.

A frequent problem occuring in parameter estimation for dynamical systems is that the objective function takes a form

$$ J(\theta, s, b, \sigma^2) = \sum_i \left[\log(2\pi\sigma_i^2) + \frac{(\bar y_i - (s_iy_i(\theta) + b_i))^2}{\sigma_i^2}\right] $$

with data $\bar y_i$, parameters $\eta = (\theta,s,b,\sigma^2)$, and ODE simulations $y(\theta)$. Here, we consider a Gaussian noise model, but also others (e.g. Laplace) are possible. Here we want to leverage that the parameter vector $\eta$ can be split into "dynamic" parameters $\theta$ that are required for simulating the ODE, and "static" parameters $s,b,\sigma^2$ that are only required to scale the simulations and formulate the objective function. As simulating the ODE usually dominates the computational complexity, one can exploit this separation of parameters by formulating an outer optimization problem in which $\theta$ is optimized, and an inner optimization problem in which $s,b,\sigma^2$ are optimized conditioned on $\theta$. This approach has shown to have superior performance to the classic approach of jointly optimizing $\eta$.

In pyPESTO, we have implemented the algorithms developed in [Loos et al.; Hierarchical optimization for the efficient parametrization of ODE models; Bioinformatics 2018](https://academic.oup.com/bioinformatics/article/34/24/4266/5053308) (covering Gaussian and Laplace noise models with gradients computed via forward sensitivity analysis) and [Schmiester et al.; Efficient parameterization of large-scale dynamic models based on relative measurements; Bioinformatics 2019](https://academic.oup.com/bioinformatics/article/36/2/594/5538985) (extending to offset parameters and adjoint sensitivity analysis).

However, the current implementation only supports:
- Gaussian (normal) noise distributions
- unbounded inner noise parameters $\sigma$
- linearly-scaled inner parameters $\eta$

Scaling $s$ and offset $b$ parameters can be bounded arbitrarily.

In the following we will demonstrate how to use hierarchical optimization, and we will compare optimization results for the following scenarios:

* Standard non-hierarchical gradient-based optimization with adjoint sensitivity analysis
* Hierarchical gradient-based optimization with analytical inner solver and adjoint sensitivity analysis
* Hierarchical gradient-based optimization with analytical inner solver and forward sensitivity analysis
* Hierarchical gradient-based optimization with numerical inner solver and adjoint sensitivity analysis

In [ ]:
import time

import amici
import fides
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import to_rgba

import pypesto
from pypesto.hierarchical.relative import (
    AnalyticalInnerSolver,
    NumericalInnerSolver,
)
from pypesto.optimize.options import OptimizeOptions
from pypesto.petab import PetabImporter

#### Note on inclusion of additional data types:
It is possible to include observables with different types of data to the same `petab_problem`. Refer to the notebooks on using [semiquantitative data](semiquantitative_data.ipynb), [ordinal data](ordinal_data.ipynb) and [censored data](censored_data.ipynb) for details on integration of other data types.

## Problem specification

We consider a version of the [Boehm et al.; Journal of Proeome Research 2014] model, modified to include scalings $s$, offsets $b$, and noise parameters $\sigma^2$.

NB: We load two PEtab problems here, because the employed standard and hierarchical optimization methods have different expectations for parameter bounds. The difference between the two PEtab problems is that the `corrected_bounds` version estimates inner noise parameters ($\sigma$) in $[0, \infty]$.

In [ ]:
# get the PEtab problem
# requires installation of
from pypesto.testing.examples import (
    get_Boehm_JProteomeRes2014_hierarchical_petab,
    get_Boehm_JProteomeRes2014_hierarchical_petab_corrected_bounds,
)

petab_problem = get_Boehm_JProteomeRes2014_hierarchical_petab()
petab_problem_hierarchical = (
    get_Boehm_JProteomeRes2014_hierarchical_petab_corrected_bounds()
)

To convert a non-hierarchical PEtab model to a hierarchical one, the observable_df, the measurement_df and the parameter_df have to be changed accordingly.
The PEtab **observable table contains** placeholders for scaling parameters $s$ (`observableParameter1_{pSTAT5A_rel,pSTAT5B_rel,rSTAT5A_rel}`), offsets $b$ (`observableParameter2_{pSTAT5A_rel,pSTAT5B_rel,rSTAT5A_rel}`), and noise parameters $\sigma^2$ (`noiseParameter1_{pSTAT5A_rel,pSTAT5B_rel,rSTAT5A_rel}`) that are overridden by the `{observable,noise}Parameters` column in the **measurement table**.

N.B.: in general, the inner parameters can appear in observable formulae directly. For example, the first observable formula in this table could be changed from `observableParameter2_pSTAT5A_rel + observableParameter1_pSTAT5A_rel * (100 * pApB + 200 * pApA * specC17) / (pApB + STAT5A * specC17 + 2 * pApA * specC17)` to `offset_pSTAT5A_rel + scaling_pSTAT5A_rel * (100 * pApB + 200 * pApA * specC17) / (pApB + STAT5A * specC17 + 2 * pApA * specC17)`.

Observable DF:

In [ ]:
from pandas import option_context

with option_context("display.max_colwidth", 400):
    display(petab_problem_hierarchical.observable_df)

Measurement DF:

In [ ]:
from pandas import option_context

with option_context("display.max_colwidth", 400):
    display(petab_problem_hierarchical.measurement_df)

Parameters to be optimized in the inner problem are specified via the PEtab parameter table by setting a value in the non-standard column `parameterType` (`offset` for offset parameters, `scaling` for scaling parameters, and `sigma` for sigma parameters). When using hierarchical optimization, the nine overriding parameters {offset,scaling,sd}_{pSTAT5A_rel,pSTAT5B_rel,rSTAT5A_rel} are to estimated in the inner problem.

In [ ]:
petab_problem_hierarchical.parameter_df

In [ ]:
# Create a pypesto problem with hierarchical optimization (`problem`)
importer = PetabImporter(
    petab_problem_hierarchical,
    hierarchical=True,
    model_name="Boehm_JProteomeRes2014_hierarchical",
)
problem = importer.create_problem(verbose=False)
# set option to compute objective function gradients using adjoint sensitivity analysis
problem.objective.amici_solver.set_sensitivity_method(
    amici.SensitivityMethod.adjoint
)

# ... and create another pypesto problem without hierarchical optimization (`problem2`)
importer2 = PetabImporter(
    petab_problem,
    hierarchical=False,
    model_name="Boehm_JProteomeRes2014_hierarchical",
)
problem2 = importer2.create_problem(verbose=False)
problem2.objective.amici_solver.set_sensitivity_method(
    amici.SensitivityMethod.adjoint
)

# Options for multi-start optimization
minimize_kwargs = {
    # number of starts for multi-start optimization
    "n_starts": 3,
    # number of processes for parallel multi-start optimization
    "engine": pypesto.engine.MultiProcessEngine(n_procs=3),
    # raise in case of failures
    "options": OptimizeOptions(allow_failed_starts=False),
    # use the Fides optimizer
    "optimizer": pypesto.optimize.FidesOptimizer(
        verbose=0, hessian_update=fides.BFGS()
    ),
}
# Set the same starting points for the hierarchical and non-hierarchical problem
startpoints = pypesto.startpoint.latin_hypercube(
    n_starts=minimize_kwargs["n_starts"],
    lb=problem2.lb_full,
    ub=problem2.ub_full,
)
# for the hierarchical problem, we only specify the outer parameters
outer_indices = [problem2.x_names.index(x_id) for x_id in problem.x_names]
problem.set_x_guesses(startpoints[:, outer_indices])
problem2.set_x_guesses(startpoints)

## Hierarchical optimization using analytical or numerical inner solver

In [ ]:
# Run hierarchical optimization using NumericalInnerSolver
start_time = time.time()
problem.objective.calculator.inner_solver = NumericalInnerSolver(
    minimize_kwargs={"n_starts": 1}
)
result_num = pypesto.optimize.minimize(problem, **minimize_kwargs)
print(f"{result_num.optimize_result.get_for_key('fval')=}")
time_num = time.time() - start_time
print(f"{time_num=}")

In [ ]:
# Run hierarchical optimization using AnalyticalInnerSolver
start_time = time.time()
problem.objective.calculator.inner_solver = AnalyticalInnerSolver()
result_ana = pypesto.optimize.minimize(problem, **minimize_kwargs)
print(f"{result_ana.optimize_result.get_for_key('fval')=}")
time_ana = time.time() - start_time
print(f"{time_ana=}")

We can visualize the estimated linear observable mapping of both the analytical and numerical approach using the `pypesto.visualize.observable_mapping.visualize_estimated_observable_mapping` routine. The routine plots all estimated linear or spline observable mappings of relative or semiquantitative observables, respectively.

In [ ]:
pypesto.visualize.observable_mapping.visualize_estimated_observable_mapping(
    pypesto_result=result_num,
    pypesto_problem=problem,
    figsize=(7, 7),
)

In [ ]:
pypesto.visualize.observable_mapping.visualize_estimated_observable_mapping(
    pypesto_result=result_ana,
    pypesto_problem=problem,
    figsize=(7, 7),
)

In [ ]:
# Waterfall plot - analytical vs numerical inner solver
pypesto.visualize.waterfall(
    [result_num, result_ana],
    legends=["Numerical-Hierarchical", "Analytical-Hierarchical"],
    size=(15, 6),
    order_by_id=True,
    colors=np.array(list(map(to_rgba, ("green", "purple")))),
)

In [ ]:
# Time comparison - analytical vs numerical inner solver
ax = plt.bar(x=[0, 1], height=[time_ana, time_num], color=["purple", "green"])
ax = plt.gca()
ax.set_xticks([0, 1])
ax.set_xticklabels(["Analytical-Hierarchical", "Numerical-Hierarchical"])
ax.set_ylabel("Computation time [s]")

## Comparison of hierarchical and non-hierarchical optimization

In [ ]:
# Run standard optimization
start_time = time.time()
result_ord = pypesto.optimize.minimize(problem2, **minimize_kwargs)
print(f"{result_ord.optimize_result.get_for_key('fval')=}")
time_ord = time.time() - start_time
print(f"{time_ord=}")

In [ ]:
# Waterfall plot - hierarchical optimization with analytical inner solver vs standard optimization
pypesto.visualize.waterfall(
    [result_ana, result_ord],
    legends=["Analytical-Hierarchical", "Non-Hierarchical"],
    order_by_id=True,
    colors=np.array(list(map(to_rgba, ("purple", "orange")))),
    size=(15, 6),
)

In [ ]:
# Time comparison - hierarchical optimization with analytical inner solver vs standard optimization
import matplotlib.pyplot as plt

ax = plt.bar(x=[0, 1], height=[time_ana, time_ord], color=["purple", "orange"])
ax = plt.gca()
ax.set_xticks([0, 1])
ax.set_xticklabels(["Analytical-Hierarchical", "Non-Hierarchical"])
ax.set_ylabel("Computation time [s]")

## Comparison of hierarchical and non-hierarchical optimization with adjoint and forward sensitivities

In [ ]:
# Run hierarchical optimization with analytical inner solver and forward sensitivities
start_time = time.time()
problem.objective.calculator.inner_solver = AnalyticalInnerSolver()
problem.objective.amici_solver.set_sensitivity_method(
    amici.SensitivityMethod_forward
)
result_ana_fw = pypesto.optimize.minimize(problem, **minimize_kwargs)
print(f"{result_ana_fw.optimize_result.get_for_key('fval')=}")
time_ana_fw = time.time() - start_time
print(f"{time_ana_fw=}")

In [ ]:
# Waterfall plot - compare all scenarios
pypesto.visualize.waterfall(
    [result_ana, result_ana_fw, result_num, result_ord],
    legends=[
        "Analytical-Hierarchical (adjoint)",
        "Analytical-Hierarchical (forward)",
        "Numerical-Hierarchical",
        "Non-Hierarchical",
    ],
    colors=np.array(list(map(to_rgba, ("purple", "blue", "green", "orange")))),
    order_by_id=True,
    size=(15, 6),
)

In [ ]:
# Time comparison of all scenarios
import matplotlib.pyplot as plt

ax = plt.bar(
    x=[0, 1, 2, 3],
    height=[time_ana, time_ana_fw, time_num, time_ord],
    color=["purple", "blue", "green", "orange"],
)
ax = plt.gca()
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(
    [
        "Analytical-Hierarchical (adjoint)",
        "Analytical-Hierarchical (forward)",
        "Numerical-Hierarchical",
        "Non-Hierarchical",
    ]
)
plt.setp(ax.get_xticklabels(), fontsize=10, rotation=75)
ax.set_ylabel("Computation time [s]")